In [15]:
import numpy as np

from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

# Load the data

The data has been collected as a rosbags. Rosbags are unpacked here and related topic decoded to dataframes or time series data format. 

In our experiment we have a differential wheeled robot that equipped with lidar, imu, a wall and another static sensor bot that equipped with lidar to measure the stopping distance of the target_bot. 

<img src="pics/gazebo_setup.png" width="400" height="400">

# Layout of the experiment!

As one can see from the above Figure, there is a wheeled robot facing a wall and another sensor bot which has the wall and the wheeled robot in its FoV. 

<img src="pics/gazebo_setup_pcl.png" width="400" height="400">

the wheeled robot will be tasked via **MockMission** to follow a straight path toward the wall. In the meantime the robot will be listening any **EmergencyStop** requests. 

Unless any **EmergencyStop** requests have been made or **MockMission** has been achieved the robot will try to correct its yaw and arrive to the desired destination.

The general system architecture can be found below, 

<img src="pics/system_architecture.png" width="400" height="400">

The simplest model of **MockMission**

<img src="pics/mock_mission_architecture.png" width="200" height="200">

Available topics can be found below:

<img src="pics/topic_names.png" width="400" height="600">

In [2]:
import os
import sys

sys.path.insert(0, os.path.abspath('./utilities'))

import utilities.converter as cnvrtr
import utilities.signal_processing as sp

# ATTENTION
Type your fullpath of your data folder and log name will be processed

In [6]:
root_folder = '/home/jpdemir/Personal/Portfolio/Robotics/System Id for Stopping Distance of a Wheeled Robot/data/'
bag_name = '2021-10-27-21-08-28.bag'

emergency_stop_topic = '/emergency_stop'
df_emergency_stop = cnvrtr.rosbag_topic_to_df(root_folder+bag_name, emergency_stop_topic)

sensorbot_laser_topic = '/myworld/laser/scan'
df_sensorbot_laser = cnvrtr.rosbag_topic_to_df(root_folder+bag_name, sensorbot_laser_topic)

[INFO]  Data folder /home/jpdemir/Personal/Portfolio/Robotics/System Id for Stopping Distance of a Wheeled Robot/data/2021-10-27-21-08-28 already exists. Not creating.
[INFO]  Data folder /home/jpdemir/Personal/Portfolio/Robotics/System Id for Stopping Distance of a Wheeled Robot/data/2021-10-27-21-08-28 already exists. Not creating.


# From Sensorbot Measurements to Wall and Robot Distances
We need to initialise range col names to be able to filter the data based on the mask we came up. Mask creation is a manual process atm, and it has been explained in the **Appendix** section.

In [8]:
# Col_filter to just select ranges related field
ranges_col_names = []
for col in df_sensorbot_laser.columns:
    if 'ranges' in col:
        ranges_col_names.append(col)

In [9]:
wall_dist = []
robot_dist = []

timestamp = []
prev_dist = 0.0

robot_mask = [560, 574] # <- came from the index_of_ranges list
wall_mask = [70, 259]

for index,row in df_sensorbot_laser.iterrows():
    timestamp.append(row["header.stamp.secs"]+row["header.stamp.nsecs"]/1e9)
    # Get range and azimuth
    robot_range_val, robot_range_azimuth = cnvrtr.range_and_azimuth_from_mask(row, ranges_col_names, robot_mask)
    # Range, Azimuth to X,Y
    robot_x, robot_y = cnvrtr.range_azimuth_to_x_y(robot_range_val, robot_range_azimuth)
    # Get dist
    robot_dist.append(sp.dist_calc(robot_x, robot_y))
    
    # Get range and azimuth
    wall_range_val, wall_range_azimuth = cnvrtr.range_and_azimuth_from_mask(row, ranges_col_names, wall_mask)
    # Range, Azimuth to X,Y
    wall_x, wall_y = cnvrtr.range_azimuth_to_x_y(wall_range_val, wall_range_azimuth)
    # Get dist
    wall_dist.append(sp.dist_calc(wall_x, wall_y))

# Polishing the data before processing
We have some fancy looking data, it is time to a bit polish them so we could detect when the robot actually stopped

In [10]:
filtered_robot_dist = sp.moving_average_filter(data=robot_dist,window_size=3)
filtered_wall_dist = sp.moving_average_filter(data=wall_dist,window_size=3)

# Calculate the speed and polish a bit more

In [11]:
robot_speed = sp.calc_speed(robot_dist, timestamp)
wall_speed = sp.calc_speed(wall_dist, timestamp)

filtered_robot_speed = sp.moving_average_filter(data=robot_speed, window_size=6)
filtered_wall_speed = sp.moving_average_filter(data=wall_speed, window_size=6)

# When the Emergency stop has been requested

## Assumptions

- Only the first **EmergencyStop** request will be taken seriously

In [12]:
emergency_stop_time = df_emergency_stop["Time"][0]
emergency_stop_idx = timestamp.index(min(timestamp, key=lambda x:abs(x-emergency_stop_time)))
print(f"Emergency stop happened at: {timestamp[emergency_stop_idx]} s.")

Emergency stop happened at: 138.829 s.


# Further polish on speed signal to detect when the robot actually stopped


In [13]:
filtered_stop_start_speed = sp.stop_start_filter(filtered_robot_speed, window_size=10)
emergency_stop_after_filtered_speed = filtered_stop_start_speed[emergency_stop_idx:]
for idx, ele in enumerate(emergency_stop_after_filtered_speed):
    if ele == 0:
        break
print(f"Robot actually stopped at: {timestamp[emergency_stop_idx + idx]} s.")
print(f"Robot stopped {timestamp[emergency_stop_idx+idx] - timestamp[emergency_stop_idx]} s afteer receiving the emergency stop request.")

Robot actually stopped at: 139.229 s.
Robot stopped 0.4000000000000057 s afteer receiving the emergency stop request.


# Sanity Check plots
If you get closer to the red and pink line you can see the difference between the emergency request and actual stop time.

In [16]:
from bokeh.layouts import row

p1 = figure(width=400, height=400, x_axis_label = "Frame idx", y_axis_label= "Distance (m)", title = "Distance vs Time Graph")

# add a circle renderer with a size, color, and alpha
p1.line(range(0, len(robot_dist)), robot_dist, legend_label = "Unfiltered Robot Distance",  color="navy", alpha=0.5)
p1.line(range(0, len(wall_dist)), wall_dist, legend_label = "Unfiltered Wall Distance", color="green", alpha=0.5)
p1.line(range(0, len(filtered_robot_dist)), filtered_robot_dist, legend_label = "Filtered Robot Distance", color="red", alpha=0.5)

p2 = figure(width=400, height=400, x_axis_label = "Frame idx", y_axis_label= "Velocity (m/s)", title = "Velocity vs Time Graph")

p2.line(range(0, len(robot_speed)), robot_speed, color="green", legend_label = "Unfiltered Speed", alpha=0.5)
p2.line(range(0, len(filtered_robot_speed)), filtered_robot_speed, color="navy", legend_label = "Filtered Speed", alpha=0.5)
#p2.line(range(0, len(speed_maf_maf)), speed_maf_maf, color="green", alpha=0.5)
p2.line(np.tile(idx+emergency_stop_idx, (1000)),np.linspace(-0.5, 0.5, num=1000), color="red", legend_label = "Robot Actually Stopped", line_width = 5,alpha=0.5)
p2.line(np.tile(emergency_stop_idx, (1000)),np.linspace(-0.5, 0.5, num=1000), color="pink", legend_label = "Emergency Stop Request", line_width = 5,alpha=0.5)

p = row(p1,p2)
# show the results
show(p)

# APPENDIX

## How to find masks for wall and robot?

In [1]:
import os
import sys

sys.path.insert(0, os.path.abspath('./utilities'))

import utilities.converter as cnvrtr
import utilities.signal_processing as sp

import math

In [2]:
data_path = '2021-10-23-19-32-36.bag'

sensorbot_laser_topic = '/myworld/laser/scan'
df_sensorbot_laser = cnvrtr.rosbag_topic_to_df(data_path, sensorbot_laser_topic)

[INFO]  Data folder 2021-10-23-19-32-36 already exists. Not creating.


In [3]:
# Col_filter to just select ranges related field
ranges_col_names = []
for col in df_sensorbot_laser.columns:
    if 'ranges' in col:
        ranges_col_names.append(col)

In [4]:
df_sensorbot_laser

,Time,header.seq,header.stamp.secs,header.stamp.nsecs,header.frame_id,angle_min,angle_max,angle_increment,time_increment,scan_time,...,intensities_710,intensities_711,intensities_712,intensities_713,intensities_714,intensities_715,intensities_716,intensities_717,intensities_718,intensities_719
0,115.725,0,115,723000000,hokuyo,-0.5,5.0,0.00765,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,115.750,1,115,748000000,hokuyo,-0.5,5.0,0.00765,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,115.776,2,115,773000000,hokuyo,-0.5,5.0,0.00765,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,115.802,3,115,799000000,hokuyo,-0.5,5.0,0.00765,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,115.825,4,115,823000000,hokuyo,-0.5,5.0,0.00765,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319,148.679,1319,148,654000000,hokuyo,-0.5,5.0,0.00765,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1320,148.680,1320,148,679000000,hokuyo,-0.5,5.0,0.00765,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1321,148.725,1321,148,704000000,hokuyo,-0.5,5.0,0.00765,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1322,148.768,1322,148,730000000,hokuyo,-0.5,5.0,0.00765,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Traverse through values and find out which ones are not inf
# sensor related info
sensor_sample_number = 720

index_of_ranges = []
for index, row in df_sensorbot_laser.iterrows():
    index_of_ranges = []
    for ranges_col_name in ranges_col_names:
        if not math.isinf(row[ranges_col_name]):
            index_of_ranges.append(ranges_col_name)
    print(f"size of the ranges: {len(index_of_ranges)}")
    if len(index_of_ranges) != sensor_sample_number:
        break

size of the ranges: 199


In [13]:
index_of_ranges

['ranges_70',
 'ranges_71',
 'ranges_72',
 'ranges_73',
 'ranges_74',
 'ranges_75',
 'ranges_76',
 'ranges_77',
 'ranges_78',
 'ranges_79',
 'ranges_80',
 'ranges_81',
 'ranges_82',
 'ranges_83',
 'ranges_84',
 'ranges_85',
 'ranges_86',
 'ranges_87',
 'ranges_88',
 'ranges_89',
 'ranges_90',
 'ranges_91',
 'ranges_92',
 'ranges_93',
 'ranges_94',
 'ranges_95',
 'ranges_96',
 'ranges_97',
 'ranges_98',
 'ranges_99',
 'ranges_100',
 'ranges_101',
 'ranges_102',
 'ranges_103',
 'ranges_104',
 'ranges_105',
 'ranges_106',
 'ranges_107',
 'ranges_108',
 'ranges_109',
 'ranges_110',
 'ranges_111',
 'ranges_112',
 'ranges_113',
 'ranges_114',
 'ranges_115',
 'ranges_116',
 'ranges_117',
 'ranges_118',
 'ranges_119',
 'ranges_120',
 'ranges_121',
 'ranges_122',
 'ranges_123',
 'ranges_124',
 'ranges_125',
 'ranges_126',
 'ranges_127',
 'ranges_128',
 'ranges_129',
 'ranges_130',
 'ranges_131',
 'ranges_132',
 'ranges_133',
 'ranges_134',
 'ranges_135',
 'ranges_136',
 'ranges_137',
 'ranges_1

# Visual inspection

Check the above parameters those will give you the mask. It is manual atm :( 

```py
# TODO(jp): Create polar image and automotize mask extraction
```

In [7]:
# mask: [min_azimuth_idx max_azimuth_idx]
robot_mask = [560, 574] # <- came from the index_of_ranges list
wall_mask = [70, 259]


robot_range_val, robot_range_azimuth = cnvrtr.range_and_azimuth_from_mask(row, ranges_col_names, robot_mask)
wall_range_val, wall_range_azimuth = cnvrtr.range_and_azimuth_from_mask(row, ranges_col_names, wall_mask)

In [8]:
robot_x,robot_y = cnvrtr.range_azimuth_to_x_y(robot_range_val, robot_range_azimuth)
wall_x,wall_y = cnvrtr.range_azimuth_to_x_y(wall_range_val, wall_range_azimuth)

In [12]:
# Plot
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
output_notebook()
# output to static HTML file
#output_file("line.html")

p = figure(width=600, height=400, x_axis_label= "x", y_axis_label = "y" , title= "Pose of wall and wheeled robot (sensorbot is the origin)")

# add a circle renderer with a size, color, and alpha
p.circle(robot_x, robot_y, size=1, color="navy", legend_label = "Robot", alpha=0.5)
p.circle(wall_x, wall_y, size=1, color="green", legend_label = "Wall",alpha=0.5)
p.diamond([0],[0], size=20, color="olive", legend_label = "Sensorbot", alpha=0.5)

# show the results
show(p)

Loading BokehJS ...